In [1]:
!unzip kcgof.zip
# !unzip independent_jobs.zip

Archive:  kcgof.zip
   creating: kcgof/
  inflating: __MACOSX/._kcgof        
  inflating: kcgof/cdata.py          
  inflating: __MACOSX/kcgof/._cdata.py  
  inflating: kcgof/plot.py           
  inflating: __MACOSX/kcgof/._plot.py  
  inflating: kcgof/config.py         
  inflating: __MACOSX/kcgof/._config.py  
  inflating: kcgof/kernel.py         
  inflating: __MACOSX/kcgof/._kernel.py  
   creating: kcgof/test/
  inflating: __MACOSX/kcgof/._test   
  inflating: kcgof/log.py            
  inflating: __MACOSX/kcgof/._log.py  
  inflating: kcgof/util.py           
  inflating: __MACOSX/kcgof/._util.py  
   creating: kcgof/ex/
  inflating: __MACOSX/kcgof/._ex     
 extracting: kcgof/__init__.py       
  inflating: __MACOSX/kcgof/.___init__.py  
  inflating: kcgof/cgoftest.py       
  inflating: __MACOSX/kcgof/._cgoftest.py  
  inflating: kcgof/glo.py            
  inflating: __MACOSX/kcgof/._glo.py  
  inflating: kcgof/cdensity.py       
  inflating: __MACOSX/kcgof/._cdensity.py  
 ex

In [2]:
!pip install git+https://github.com/wittawatj/kernel-gof.git
!pip install dill

  Cloning https://github.com/wittawatj/kernel-gof.git to /tmp/pip-req-build-tsyuc4ep
  Running command git clone --filter=blob:none --quiet https://github.com/wittawatj/kernel-gof.git /tmp/pip-req-build-tsyuc4ep
  Resolved https://github.com/wittawatj/kernel-gof.git to commit 039a95ed9d8062e283da6bd051b7161a190b4876
  Preparing metadata (setup.py) ... done
  Created wheel for kgof: filename=kgof-0.1.0-py3-none-any.whl size=44899 sha256=35a956c77aed80717de30d2c486bda72644ee6fc2941f40ad71f33aee99e7c03
  Stored in directory: /tmp/pip-ephem-wheel-cache-u57saxqo/wheels/64/85/bb/4f8511deef3c71cea175435fa3a42619797129c8842e017238
Successfully built kgof
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.9 MB/s eta 0:00:00


In [3]:
import numpy as np
import torch

from IPython.display import clear_output

import kcgof
import kcgof.log as klog
import kcgof.util as util
import kcgof.cdensity as cden
import kcgof.cdata as cdat
import kcgof.cgoftest as cgof
import kcgof.kernel as ker
import kcgof.plot as plot

In [ ]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [26]:
def simple_gaussian_model(X, Y):

    slope = torch.tensor([1.0, 1.0, 1.0])
    center = 0.8*torch.ones(1,3)
    variance = torch.tensor(1.0) + 10*torch.exp( -0.5*torch.sum( (X - center)**2, dim=1 )/0.8**2 )

    linear = torch.matmul(X, slope)

    return -(Y.view(-1)-linear)**2/(2.0*variance) - (0.5) * torch.log(variance)

In [27]:
dx = 3
dy = 1

# Model p(y|x)
p = cden.from_log_den(dx, dy, simple_gaussian_model)

B = 200
sample_size = [50, 100, 150, 200, 250, 300]
TypeI_vec = []
Power_vec = []

In [28]:
for n in sample_size:
  TypeI_error = 0
  for r in range(B):
    seed_everything(r)
    real_slope = torch.tensor([1.0, 1.0, 1.0])

    # sample size
    # n = 2000
    X = torch.tensor(np.random.randn(n, dx), dtype=torch.float32)
    linear = torch.matmul(X, real_slope)

    # true variance
    center = 0.8*torch.ones(1,dx)
    real_variance = torch.tensor(1.0) + 10*torch.exp( -0.5*torch.sum( (X - center)**2, dim=1 )/0.8**2 )

    output_noise = torch.randn(n)*torch.sqrt(real_variance)
    Y = linear + output_noise
    Y = Y.view(n, dy).type(torch.float32)

    sigx = util.pt_meddistance(X)
    sigy = util.pt_meddistance(Y)

    # k = kernel on X. PTKGauss takes width **squared**.
    k = ker.PTKGauss(sigma2=sigx**2/2)

    # l = kernel on Y
    l = ker.PTKGauss(sigma2=sigy**2/2)

    kcsdtest = cgof.KCSDTest(p, k, l, alpha=0.05, n_bootstrap=500)
    result = kcsdtest.perform_test(X, Y)

    TypeI_error += result['h0_rejected']

    print (r)

  clear_output(wait=True)

  TypeI_vec.append(TypeI_error/B)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [29]:
TypeI_vec

[0.06, 0.075, 0.085, 0.06, 0.065, 0.04]

In [30]:
for n in sample_size:
  Power = 0
  for r in range(B):
    seed_everything(r)
    real_slope = torch.tensor([1.0, 1.0, 1.0])

    # sample size
    # n = 2000
    X = torch.tensor(np.random.randn(n, dx), dtype=torch.float32)
    linear = torch.matmul(X, real_slope)

    # true variance
    real_variance = torch.tensor(1.0)

    output_noise = torch.randn(n)*torch.sqrt(real_variance)
    Y = linear + output_noise
    Y = Y.view(n, dy).type(torch.float32)

    sigx = util.pt_meddistance(X)
    sigy = util.pt_meddistance(Y)

    # k = kernel on X. PTKGauss takes width **squared**.
    k = ker.PTKGauss(sigma2=sigx**2/2)

    # l = kernel on Y
    l = ker.PTKGauss(sigma2=sigy**2/2)

    kcsdtest = cgof.KCSDTest(p, k, l, alpha=0.05, n_bootstrap=500)
    result = kcsdtest.perform_test(X, Y)

    Power += result['h0_rejected']

    print (r)

  clear_output(wait=True)

  Power_vec.append(Power/B)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [31]:
Power_vec

[0.22, 0.39, 0.615, 0.8, 0.93, 0.96]

In [32]:
import pandas as pd

df = pd.DataFrame({'sample_size': sample_size, 'TypeI_error': TypeI_vec, 'Power': Power_vec})

df.to_csv('results.csv', index=False)